### Imports

In [1]:
# basic imports
import os, random
import pandas as pd
import numpy as np
import datetime as dt
import pandas_ta as ta
from pathlib import Path
import yfinance as yf
import math

# warnings
import warnings
warnings.filterwarnings('ignore')

# plotting & outputs
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn')

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# # Machine info & package version
from watermark import watermark
%load_ext watermark
%watermark -a "Siqi He" -u -d -v -m -iv  

Author: Siqi He

Last updated: 2024-01-02

Python implementation: CPython
Python version       : 3.8.18
IPython version      : 8.12.2

Compiler    : Clang 16.0.6 
OS          : Darwin
Release     : 23.0.0
Machine     : arm64
Processor   : arm
CPU cores   : 8
Architecture: 64bit

numpy      : 1.22.4
seaborn    : 0.13.0
tensorflow : 2.13.1
pandas_ta  : 0.3.14b0
keras_tuner: 1.3.5
yfinance   : 0.2.33
matplotlib : 3.7.2
pandas     : 1.5.3



###  Global variables

In [2]:
THRESH = 0.1               # Threshold for positive / negative returns

TICKER_NAME = "tesco"      # name of ticker in English
TICKER = "TSCO.L"          # main ticker ID
TICKER_PERIOD = '1d'       # main ticker period
FX_TICKER = "GBPUSD=X"     # currency ticker ID
FX_TICKER_PERIOD = '1d'    # currency ticker period
INDEX_TICKER = "^FTSE"     # index ticker ID
INDEX_TICKER_PERIOD = '1d' # index ticker period

CPI_DATA_PATH = "data/cpi_mom.csv"  # data file path for storing cpi data
FF_DATA_PATH = "data/europe_3_factors_daily.csv"  # data file path for storing Fama-French factors

START = '2014-01-01'       # study period start date
END = '2023-12-25'         # study period end date

### Functions for data extraction

In [3]:
# function for data extraction
def getdata(ticker='TSCO.L', period='1d', start='2010-01-01', end='2015-12-31', progress=False):
    
    '''
    Pulls YahooFinance ticker data for specified period and frequency.

    Keyword arguments:
    ticker -- Str: The ID of the ticker (default TSCO.L)
    period -- Str: The frequency of data requested (default 1d)
    start -- Str:The start date of the period requested in format 'YYYY-MM-DD' (default 2010-01-01)
    end -- Str: The end date of the period requested in format 'YYYY-MM-DD' (default 2015-12-31)
    progress -- Bool: Print progress bar (default False)
    '''
    
    df = yf.download(ticker, period=period, 
                   start=start, end=end, progress=progress)
    
    return df

def get_matching_period_ticker(data, ticker='GBPUSD=X', period='1d', columns=['Close']):
    
    '''
    Pulls closing rate a ticker based on the start and end date of specified dataset.

    Keyword arguments:
    ticker -- Str: The ID of the ticker (default 'GBPUSD=X')
    period -- Str: The frequency of data requested (default '1d')
    columns -- List: The columns in ticker data to keep (default ['Close'], All options: 'Open', 'Close', 'High', 'Low', 'Volume')
    data -- Pandas Dataframe: the reference timeseries dataframe, where the index column timestamps stores the total period to extract new ticker
    '''
    
    # fetch rate from yahoofinance
    matching_ticker = getdata(ticker=ticker, period=period, start=data.index[0].strftime('%Y-%m-%d'), end=(data.index[-1] + dt.timedelta(days=1)).strftime('%Y-%m-%d'))[columns]

    # Drop the "Close" column
    matching_ticker.drop("Close", axis=1, inplace=True)

    # Rename the "Adj Close" column to "Close"
    matching_ticker.rename(columns={"Adj Close": "Close"}, inplace=True)
    
    # rename all columns with ticker prefix 
    [matching_ticker.rename(columns={column: f"{ticker}_{column}"}, inplace=True) for column in columns]

    return matching_ticker

def merge(data, new_data, fill='none'):

    '''
    Merge new data to existing dataset, with the option to forward fill or backward fill mull values. 

    '''
    # Join new data to data
    data = data.join(new_data, how="left")

    # Fill values as passed in argument
    if fill=='forward':
        data = data.ffill(axis=1)
    if fill=='backward':
        data = data.bfill(axis=1)
    if fill=='none':
        pass
        
    return data

### Fetch main ticker data

In [4]:
# Fetch main ticker data from YahooFinance
main = getdata(ticker=TICKER, period=TICKER_PERIOD, start=START, end=END)

# View main ticker data
main

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,425.029785,428.259796,415.783112,423.509796,272.076355,5710683
2014-01-03,419.836395,421.656586,416.606506,418.633087,268.943451,10305400
2014-01-06,420.533112,422.813110,418.189789,420.786407,270.326782,10974050
2014-01-07,422.179810,424.269806,418.443115,420.216400,269.960571,13577058
2014-01-08,424.333099,428.069794,411.033112,415.846497,267.153259,27819606
...,...,...,...,...,...,...
2023-12-18,282.799988,284.899994,281.100006,283.500000,283.500000,12033304
2023-12-19,284.200012,286.899994,281.399994,286.100006,286.100006,79752994
2023-12-20,290.700012,292.000000,284.399994,288.600006,288.600006,16327513


### Fetch sterling-dollar rate

In [5]:
# Fetch sterling-dollar rate from YahooFinance
sterlingdollar = getdata(ticker=FX_TICKER, period=TICKER_PERIOD, start=START, end=END)

# Keep adjusted closing price column 
sterlingdollar.drop(["Close", "Open", "High", "Low", "Volume"], axis=1, inplace=True)
sterlingdollar.rename(columns={"Adj Close": f"{FX_TICKER}_Close"}, inplace=True)

# replace price with percentage change
sterlingdollar = sterlingdollar.pct_change() * 100

# view data sterling-dollar daily adjusted rate
sterlingdollar

,GBPUSD=X_Close
Date,
2014-01-01,NaN
2014-01-02,0.101138
2014-01-03,-0.801011
2014-01-06,-0.211719
2014-01-07,0.029553
...,...
2023-12-18,-0.775551
2023-12-19,-0.161942
2023-12-20,0.639181


### Fetch FTSE100 Closing Rate

In [6]:
# Fetch FTSE100 daily price data from YahooFinance
FTSE = getdata(ticker=INDEX_TICKER, period=TICKER_PERIOD, start=START, end=END)

# Get adjusted closing price column
FTSE.drop(["Close", "Open", "High", "Low", "Volume"], axis=1, inplace=True)
FTSE.rename(columns={"Adj Close": f"{INDEX_TICKER}_Close"}, inplace=True)

# replace Close price with percentage change
FTSE = FTSE.pct_change() * 100

# view FTSE100 daily adjusted closing price
FTSE

,^FTSE_Close
Date,
2014-01-02,NaN
2014-01-03,0.190540
2014-01-06,0.000000
2014-01-07,0.368458
2014-01-08,-0.498856
...,...
2023-12-18,0.502879
2023-12-19,0.308622
2023-12-20,1.017285


### Merge data and convert to CSV

In [7]:
final = merge(merge(main, sterlingdollar, fill='none'), FTSE, fill='none')
final

,Open,High,Low,Close,Adj Close,Volume,GBPUSD=X_Close,^FTSE_Close
Date,,,,,,,,
2014-01-02,425.029785,428.259796,415.783112,423.509796,272.076355,5710683,0.101138,NaN
2014-01-03,419.836395,421.656586,416.606506,418.633087,268.943451,10305400,-0.801011,0.190540
2014-01-06,420.533112,422.813110,418.189789,420.786407,270.326782,10974050,-0.211719,0.000000
2014-01-07,422.179810,424.269806,418.443115,420.216400,269.960571,13577058,0.029553,0.368458
2014-01-08,424.333099,428.069794,411.033112,415.846497,267.153259,27819606,-0.108246,-0.498856
...,...,...,...,...,...,...,...,...
2023-12-18,282.799988,284.899994,281.100006,283.500000,283.500000,12033304,-0.775551,0.502879
2023-12-19,284.200012,286.899994,281.399994,286.100006,286.100006,79752994,-0.161942,0.308622
2023-12-20,290.700012,292.000000,284.399994,288.600006,288.600006,16327513,0.639181,1.017285


### Export to CSV - 
<font color='red'><b>Do NOT rerun as the YFinance package is unstable and retrieves data with rounding errors with each different pull.</b></font>

In [8]:
# output data to csv
# final.to_csv("data/tesco_fx_index_data.csv")